In [63]:
# Step 0. Load libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.formula.api as smf
import folium
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import  metrics

In [64]:
#https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv

df_raw = pd.read_csv('../data/raw/titanic_train.csv', index_col=0)


In [65]:
df_raw.shape

(891, 11)

In [66]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [67]:
df = df_raw.copy()

Como existe desbalance me tengo que fijar más en las medidas de f1 y no tanto en accuracy

In [68]:
#Vemos si está desbalanceado
df['Survived'].value_counts()/len(df['Survived'])

0    0.616162
1    0.383838
Name: Survived, dtype: float64

Algoritmos que se podrían usar:
Decision tree, logistic regression, entre otros.
Algoritmos más complejos necesitan más datos.
Criterio para analizar random forest es más complejo para interpretar, deberíamos de analizar cada árbol y realizar el promedio.

In [69]:
#Todos los nombres son diferentes
df['Name'].nunique()

891

In [70]:
df=df.drop(columns=['Name'])

In [71]:
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [72]:
#cambiamos Sex por números
df['Sex']=df['Sex'].map({'male':1,'female':0})

In [73]:
df['Sex']

PassengerId
1      1
2      0
3      0
4      0
5      1
      ..
887    1
888    0
889    0
890    1
891    1
Name: Sex, Length: 891, dtype: int64

In [74]:
df.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
650,1,3,0,23.0,0,0,CA. 2314,7.5500,NaN,S
765,0,3,1,16.0,0,0,347074,7.7750,NaN,S
359,1,3,0,NaN,0,0,330931,7.8792,NaN,Q
531,1,2,0,2.0,1,1,26360,26.0000,NaN,S
821,1,1,0,52.0,1,1,12749,93.5000,B69,S
109,0,3,1,38.0,0,0,349249,7.8958,NaN,S
396,0,3,1,22.0,0,0,350052,7.7958,NaN,S
507,1,2,0,33.0,0,2,26360,26.0000,NaN,S
773,0,2,0,57.0,0,0,S.O./P.P. 3,10.5000,E77,S


In [75]:
df['Ticket'].nunique()

681

In [76]:
df=df.drop(columns=['Ticket'])

In [77]:
df['Cabin'].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

In [78]:
df['Cabin'].nunique()

147

In [79]:
df=df.drop(columns=['Cabin'])

In [80]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [81]:
df['Embarked']=df['Embarked'].map({'S':2,'C':1,'Q':0})

In [82]:
df['Embarked']

PassengerId
1      2.0
2      1.0
3      2.0
4      2.0
5      2.0
      ... 
887    2.0
888    2.0
889    2.0
890    1.0
891    0.0
Name: Embarked, Length: 891, dtype: float64

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 62.6 KB


In [84]:
mean_age = df['Age'].mean()
print(mean_age)

df['Age'][np.isnan(df['Age'])] = mean_age


29.69911764705882


/tmp/ipykernel_6015/1682399611.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'][np.isnan(df['Age'])] = mean_age


In [85]:
df['Embarked'].value_counts()

2.0    644
1.0    168
0.0     77
Name: Embarked, dtype: int64

In [86]:
df['Embarked'][np.isnan(df['Embarked'])] = 2

/tmp/ipykernel_6015/2528881676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'][np.isnan(df['Embarked'])] = 2


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    float64
dtypes: float64(3), int64(5)
memory usage: 62.6 KB


In [88]:
#Dividimos el dataset
X=df.drop(columns=['Survived'])
y=df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=520, test_size=0.2) #darle menos datos de entrenamiento para que no haya overfit
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=520, test_size=0.3)

leakage: nivel de correlación de las variables

In [89]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22.0,1,0,7.2500,2.0
2,1,0,38.0,1,0,71.2833,1.0
3,3,0,26.0,0,0,7.9250,2.0
4,1,0,35.0,1,0,53.1000,2.0
5,3,1,35.0,0,0,8.0500,2.0


In [90]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [91]:
#nos da los parámetros
#RandomForestClassifier?


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [92]:
modelo=RandomForestClassifier(n_estimators=50, random_state=145) # Le damos menos árboles para que no haya overfit

In [93]:
modelo.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50, random_state=145)

In [94]:
y_train_pred =modelo.predict(X_train)
y_test_pred =modelo.predict(X_test)

In [95]:
#Usamos una métrica para ver las diferencias

train_report =metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       456
           1       0.98      0.97      0.98       256

    accuracy                           0.98       712
   macro avg       0.98      0.98      0.98       712
weighted avg       0.98      0.98      0.98       712



In [96]:
test_report =metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.78      0.90      0.84        93
           1       0.87      0.72      0.79        86

    accuracy                           0.82       179
   macro avg       0.83      0.81      0.81       179
weighted avg       0.82      0.82      0.81       179



f1 score del train es 1 y del test es 0.76, al algoritmo esta memorizando la data.

CAMBIAMOS LOS DATOS PARA AJUSTAR

In [97]:
modelo=RandomForestClassifier(n_estimators=5, random_state=42, max_leaf_nodes=3)

In [98]:
modelo.fit(X_train,y_train)

RandomForestClassifier(max_leaf_nodes=3, n_estimators=5, random_state=42)

In [99]:
y_train_pred =modelo.predict(X_train)
y_test_pred =modelo.predict(X_test)

In [100]:
train_report =metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       456
           1       0.75      0.66      0.70       256

    accuracy                           0.80       712
   macro avg       0.79      0.77      0.78       712
weighted avg       0.80      0.80      0.80       712



In [101]:
test_report =metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.74      0.90      0.82        93
           1       0.86      0.66      0.75        86

    accuracy                           0.79       179
   macro avg       0.80      0.78      0.78       179
weighted avg       0.80      0.79      0.78       179

